In [82]:
import pandas as pd
from glob import glob
import numpy as np

In [44]:
## Getting the key information
main_data = pd.read_csv('D:/Projects/fma/data/fma_metadata/tracks.csv',header=None)
columns = main_data.iloc[:3,:].fillna(value='').apply(lambda a : a +' ').sum().apply(lambda a : a.strip()).values
main_data = main_data.iloc[3:]
main_data.columns = columns
main_data.index = main_data['track_id'].astype(int)
main_data = main_data[['track title', 'track genres', 'artist name', 'artist id','track bit_rate','track duration']]

## Adding Path of songs
songs = glob('D:/Projects/fma/data/fma_medium/*/*.mp3')
songs = {int(i.split('\\')[-1].split('.')[0]):i for i in songs}
songs = pd.DataFrame(songs,index=[0]).T
songs.columns = ['file_path']

C:\Users\GP65\AppData\Local\Temp\ipykernel_25584\1483770685.py:2: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  main_data = pd.read_csv('D:/Projects/fma/data/fma_metadata/tracks.csv',header=None)


In [120]:
full_dataframe = pd.merge(main_data,songs,left_index=True,right_index=True,how='inner',sort=True)
full_dataframe.index.name = 'track_id'
full_dataframe['instrumental_path'] = [None] * len(full_dataframe)
full_dataframe['vocal_path'] = [None] * len(full_dataframe)
full_dataframe.to_csv('D:/Projects/Orpheus_ai/DataSet/main_dataframe.csv')

## Spleeter Test

In [31]:
import spleeter
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter

In [ ]:
separator = Separator('spleeter:2stems')

In [40]:
audio_loader = AudioAdapter.default()
sample_rate = 256000
waveform, _ = audio_loader.load('D:/Projects/fma/data/fma_medium\\000\\000002.mp3', sample_rate=sample_rate)
prediction = separator.separate(waveform)

In [66]:
separator.separate_to_file('D:/Projects/fma/data/fma_medium\\000\\000002.mp3', 'D:/Projects/Orpheus_ai/DataSet/Samples/',
                          filename_format='{instrument}/{filename}.{codec}',
                          bitrate='128k',
                          duration=30,
                          codec=spleeter.audio.Codec.MP3)

## Vocal Spearation Pipeline

In [69]:
import spleeter
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter
from tqdm.notebook import tqdm
## Separator and Loader
separator = Separator('spleeter:2stems')
audio_loader = AudioAdapter.default()

In [127]:
full_dataframe = pd.read_csv('D:/Projects/Orpheus_ai/DataSet/main_dataframe.csv',index_col=['track_id'])

In [137]:
for r in tqdm(full_dataframe[full_dataframe['instrumental_path'].isna()].reset_index().values):
    try:
        index = r[0]
        path = r[-3]
        name = path.split('\\')[-1]
        
        ## Separating vocals from instrumental
        separator.separate_to_file(path, 'D:/Projects/Orpheus_ai/DataSet/Sep_Dataset/',
                              filename_format='{instrument}/{filename}.{codec}',
                              bitrate='128k',
                              duration=30,
                              codec=spleeter.audio.Codec.MP3)
        
        ## Saving Paths
        full_dataframe.loc[index,'instrumental_path'] = f"D:/Projects/Orpheus_ai/DataSet/Sep_Dataset/accompaniment/{name}"
        full_dataframe.loc[index,'vocal_path'] = f"D:/Projects/Orpheus_ai/DataSet/Sep_Dataset/vocals/{name}"
        full_dataframe.to_csv('D:/Projects/Orpheus_ai/DataSet/main_dataframe.csv')
        
    except Exception as e:
        print(f"Exception in {path} -- {e}")
        

  0%|          | 0/21629 [00:00<?, ?it/s]

Exception in D:/Projects/fma/data/fma_medium\001\001486.mp3 -- An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.4.1-full_build-www.gyan.dev Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid

Exception in D:/Projects/fma/data/fma_medium\098\098558.mp3 -- An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.4.1-full_build-www.gyan.dev Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid

Exception in D:/Projects/fma/data/fma_medium\099\099134.mp3 -- An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.4.1-full_build-www.gyan.dev Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid

Exception in D:/Projects/fma/data/fma_medium\127\127336.mp3 -- An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.4.1-full_build-www.gyan.dev Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid